In [1]:
import cv2
import pickle
import os.path
import numpy as np
import imutils
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.models import Sequential, load_model
from keras.layers import LeakyReLU, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from keras.backend.tensorflow_backend import set_session
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Using TensorFlow backend.


In [3]:
def resize_to_fit(image, width, height):
    # grab the dimensions of the image, then initialize the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along the width
    if w > h:
        image = imutils.resize(image, width=width)
    # otherwise, the height is greater than the width so resize along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW, cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image

In [4]:
# Variables

models = r"D:\Python\Jupyter\models"
training = r"D:\Python\Jupyter\training"
testing = r"D:\Python\Jupyter\testing"
model_name = "model_1"
data = []
labels = []
pixel_size = 64

vertical = ['bedroom', 'Forest', 'Insidecity', 'kitchen', 'livingroom', 'Office', 'store', 'Street', 'Suburb', 'TallBuilding']
horizontal = ['Coast', 'Highway', 'industrial', 'Mountain', 'OpenCountry']

In [5]:
# Read the images

for folder in os.listdir(training):
    print("Reading from:", folder)
    buffer = []
    
    # Grab the name of the letter based on the folder it was in
    label = folder
    for image_file in paths.list_images(os.path.join("/", training, folder)):
        # Load the image and convert it to grayscale
        
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize the letter so it fits in a pixel box
        image = resize_to_fit(image, pixel_size, pixel_size)

        # Add a third channel dimension to the image to make Keras happy
        image = np.expand_dims(image, axis=2)

        # Add the image and it's label to our training data
        data.append(image)
        labels.append(label)
        
        # Data augmenntation
        
        # 1. Flip
        image_flip = cv2.flip(image, 1)
        image_flip = np.expand_dims(image_flip, axis=2)
        
        # Add the image and it's label to our training data
        data.append(image_flip)
        labels.append(label)
        
        buffer.append(image)
        labels.append(label)
        
    list1 = []
    list2 = []
    if label in vertical:
        for x in buffer:
            x1,x2 = np.split(x, 2, axis = 0)
            list1.append(x1)
            list2.append(x2)
        
        np.random.shuffle(list1)
        np.random.shuffle(list2)
        
        listZip = zip(list1, list2)
        listMerge = list(map(lambda x: np.concatenate(x, axis = 0), listZip))
        data += listMerge
        
    elif label in horizontal:
        for x in buffer:
            x1,x2 = np.split(x, 2, axis = 1)
            list1.append(x1)
            list2.append(x2)
        
        np.random.shuffle(list1)
        np.random.shuffle(list2)
        listZip = zip(list1, list2)
        
        listMerge = list(map(lambda x: np.concatenate(x, axis = 1), listZip))
        data += listMerge
        
    else:
        print("ERROR: wrong label")

# Print thhe numbers of read images
print(len(data), "Images read")

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

Reading from: bedroom
Reading from: Coast
Reading from: Forest
Reading from: Highway
Reading from: industrial
Reading from: Insidecity
Reading from: kitchen
Reading from: livingroom
Reading from: Mountain
Reading from: Office
Reading from: OpenCountry
Reading from: store
Reading from: Street
Reading from: Suburb
Reading from: TallBuilding
4500 Images read


In [4]:
#CNN
model = Sequential()

model.add(Conv2D(100, (5, 5), padding="same", input_shape=(pixel_size, pixel_size, 1), activation="linear"))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(50, (5, 5), padding="same", activation="linear"))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(500, activation="linear"))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(15, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [5]:
# Training phase

skf = StratifiedKFold(n_splits=5, shuffle=True)
for i, (train_index, val_index) in enumerate(skf.split(data, labels)):
    # Prepare the data
    X_train, Y_train = data[train_index], labels[train_index]
    X_val, Y_val = data[val_index], labels[val_index]
    
    # To make Keras happy
    lb = LabelBinarizer().fit(Y_train)
    Y_train = lb.transform(Y_train)
    Y_val = lb.transform(Y_val)
    
    # Train the model
    print("Fold:", i)
    model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=64, epochs=3, verbose=1)

Fold: 0
Train on 3600 samples, validate on 900 samples
Epoch 1/3
3600/3600 [==============================] - 77s 21ms/step - loss: 2.4739 - acc: 0.2075 - val_loss: 2.5317 - val_acc: 0.2356
Epoch 2/3
3600/3600 [==============================] - 76s 21ms/step - loss: 1.6641 - acc: 0.4708 - val_loss: 1.4805 - val_acc: 0.5144
Epoch 3/3
3600/3600 [==============================] - 74s 20ms/step - loss: 1.0345 - acc: 0.6617 - val_loss: 1.1652 - val_acc: 0.6111
Fold: 1
Train on 3600 samples, validate on 900 samples
Epoch 1/3
3600/3600 [==============================] - 75s 21ms/step - loss: 0.7239 - acc: 0.7556 - val_loss: 0.6933 - val_acc: 0.7844
Epoch 2/3
3600/3600 [==============================] - 75s 21ms/step - loss: 0.4316 - acc: 0.8656 - val_loss: 0.5022 - val_acc: 0.8322
Epoch 3/3
3600/3600 [==============================] - 76s 21ms/step - loss: 0.2327 - acc: 0.9308 - val_loss: 0.5855 - val_acc: 0.8067
Fold: 2
Train on 3600 samples, validate on 900 samples
Epoch 1/3
3600/3600 [====

In [21]:
# Save the model

model.save(os.path.join("/", models, model_name + ".hdf5"))

In [9]:
# Load

model = load_model(os.path.join("/", models, "cross-validation_5f3e.hdf5"))

X_test = data
lb = LabelBinarizer().fit(labels)
Y_test = lb.transform(labels)

In [6]:
# Accuracy

scores = model.evaluate(X_test, Y_test, batch_size=64, verbose=1)
print("\nAccuracy: %.4f%%\n\n"%(scores[1]*100))

4500/4500 [==============================] - 26s 6ms/step

Accuracy: 98.8667%




In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 100)       2600      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 100)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 100)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 50)        125050    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 50)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 50)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
__________

In [8]:
# Testing on training data

allSum = 0
for folder in os.listdir(training):
    sum = 0
    for image_file in paths.list_images(os.path.join("/", training, folder)):
        # Load the image and convert it to grayscale
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize the letter so it fits in a pixel box
        image = resize_to_fit(image, pixel_size, pixel_size)

        # Turn the single image into a 4d list of images to make Keras happy
        image = np.expand_dims(image, axis=2)
        image = np.expand_dims(image, axis=0)

        # Grab the name of the letter based on the folder it was in
        label = folder

        # Add the image and it's label to our training data
        prediction = model.predict(image)
        prediction = lb.inverse_transform(prediction)[0]
        
        # Calculate accuracy
        if prediction == label:
            sum +=1

    print(folder, sum/100)
    allSum += sum

print(allSum/1500)

bedroom 0.94
Coast 0.62
Forest 0.81
Highway 0.63
industrial 0.72
Insidecity 0.9
kitchen 0.92
livingroom 0.86
Mountain 0.84
Office 0.56
OpenCountry 0.59
store 1.0
Street 0.94
Suburb 0.99
TallBuilding 0.94
0.8173333333333334


In [ ]:
# Testing on testing data

for image_file in paths.list_images(testing):
    # Load the image and convert it to grayscale
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize the images so it fits in a pixel box
    image = resize_to_fit(image, pixel_size, pixel_size)

    # Turn the single image into a 4d list of images to make Keras happy
    image = np.expand_dims(image, axis=2)
    image = np.expand_dims(image, axis=0)

    # Ask the neural network to make a prediction
    prediction = model.predict(image)
    prediction = lb.inverse_transform(prediction)[0]
    
    # Print prediction
    print(image_file, prediction)

D:\Python\Jupyter\testing\0.jpg store
D:\Python\Jupyter\testing\1.jpg Insidecity
D:\Python\Jupyter\testing\10.jpg Coast
D:\Python\Jupyter\testing\100.jpg industrial
D:\Python\Jupyter\testing\1000.jpg store
D:\Python\Jupyter\testing\1001.jpg livingroom
D:\Python\Jupyter\testing\1002.jpg Office
D:\Python\Jupyter\testing\1003.jpg kitchen
D:\Python\Jupyter\testing\1004.jpg store
D:\Python\Jupyter\testing\1005.jpg OpenCountry
D:\Python\Jupyter\testing\1006.jpg TallBuilding
D:\Python\Jupyter\testing\1007.jpg industrial
D:\Python\Jupyter\testing\1008.jpg livingroom
D:\Python\Jupyter\testing\1009.jpg store
D:\Python\Jupyter\testing\101.jpg store
D:\Python\Jupyter\testing\1010.jpg store
D:\Python\Jupyter\testing\1011.jpg store
D:\Python\Jupyter\testing\1012.jpg store
D:\Python\Jupyter\testing\1013.jpg industrial
D:\Python\Jupyter\testing\1014.jpg Suburb
D:\Python\Jupyter\testing\1015.jpg Mountain
D:\Python\Jupyter\testing\1016.jpg kitchen
D:\Python\Jupyter\testing\1017.jpg livingroom
D:\Python\

D:\Python\Jupyter\testing\1185.jpg Mountain
D:\Python\Jupyter\testing\1186.jpg kitchen
D:\Python\Jupyter\testing\1187.jpg store
D:\Python\Jupyter\testing\1188.jpg Highway
D:\Python\Jupyter\testing\1189.jpg store
D:\Python\Jupyter\testing\119.jpg bedroom
D:\Python\Jupyter\testing\1190.jpg store
D:\Python\Jupyter\testing\1191.jpg kitchen
D:\Python\Jupyter\testing\1192.jpg Forest
D:\Python\Jupyter\testing\1193.jpg Mountain
D:\Python\Jupyter\testing\1194.jpg OpenCountry
D:\Python\Jupyter\testing\1195.jpg bedroom
D:\Python\Jupyter\testing\1196.jpg store
D:\Python\Jupyter\testing\1197.jpg store
D:\Python\Jupyter\testing\1198.jpg livingroom
D:\Python\Jupyter\testing\1199.jpg Coast
D:\Python\Jupyter\testing\12.jpg TallBuilding
D:\Python\Jupyter\testing\120.jpg store
D:\Python\Jupyter\testing\1200.jpg Street
D:\Python\Jupyter\testing\1201.jpg kitchen
D:\Python\Jupyter\testing\1202.jpg industrial
D:\Python\Jupyter\testing\1203.jpg store
D:\Python\Jupyter\testing\1204.jpg Suburb
D:\Python\Jupyter

D:\Python\Jupyter\testing\1371.jpg bedroom
D:\Python\Jupyter\testing\1372.jpg OpenCountry
D:\Python\Jupyter\testing\1373.jpg bedroom
D:\Python\Jupyter\testing\1374.jpg Mountain
D:\Python\Jupyter\testing\1375.jpg bedroom
D:\Python\Jupyter\testing\1376.jpg livingroom
D:\Python\Jupyter\testing\1377.jpg store
D:\Python\Jupyter\testing\1378.jpg OpenCountry
D:\Python\Jupyter\testing\1379.jpg store
D:\Python\Jupyter\testing\138.jpg store
D:\Python\Jupyter\testing\1380.jpg livingroom
D:\Python\Jupyter\testing\1381.jpg Suburb
D:\Python\Jupyter\testing\1382.jpg TallBuilding
D:\Python\Jupyter\testing\1383.jpg Suburb
D:\Python\Jupyter\testing\1384.jpg OpenCountry
D:\Python\Jupyter\testing\1385.jpg kitchen
D:\Python\Jupyter\testing\1386.jpg TallBuilding
D:\Python\Jupyter\testing\1387.jpg store
D:\Python\Jupyter\testing\1388.jpg Mountain
D:\Python\Jupyter\testing\1389.jpg store
D:\Python\Jupyter\testing\139.jpg store
D:\Python\Jupyter\testing\1390.jpg store
D:\Python\Jupyter\testing\1391.jpg bedroom

D:\Python\Jupyter\testing\1553.jpg Mountain
D:\Python\Jupyter\testing\1554.jpg TallBuilding
D:\Python\Jupyter\testing\1555.jpg store
D:\Python\Jupyter\testing\1556.jpg OpenCountry
D:\Python\Jupyter\testing\1557.jpg kitchen
D:\Python\Jupyter\testing\1558.jpg Mountain
D:\Python\Jupyter\testing\1559.jpg Suburb
D:\Python\Jupyter\testing\156.jpg bedroom
D:\Python\Jupyter\testing\1560.jpg bedroom
D:\Python\Jupyter\testing\1561.jpg Street
D:\Python\Jupyter\testing\1562.jpg OpenCountry
D:\Python\Jupyter\testing\1563.jpg Highway
D:\Python\Jupyter\testing\1564.jpg store
D:\Python\Jupyter\testing\1565.jpg Mountain
D:\Python\Jupyter\testing\1566.jpg kitchen
D:\Python\Jupyter\testing\1567.jpg bedroom
D:\Python\Jupyter\testing\1568.jpg bedroom
D:\Python\Jupyter\testing\1569.jpg bedroom
D:\Python\Jupyter\testing\157.jpg Office
D:\Python\Jupyter\testing\1570.jpg Coast
D:\Python\Jupyter\testing\1571.jpg Forest
D:\Python\Jupyter\testing\1572.jpg TallBuilding
D:\Python\Jupyter\testing\1573.jpg kitchen
D:

D:\Python\Jupyter\testing\1729.jpg store
D:\Python\Jupyter\testing\173.jpg kitchen
D:\Python\Jupyter\testing\1730.jpg TallBuilding
D:\Python\Jupyter\testing\1731.jpg industrial
D:\Python\Jupyter\testing\1732.jpg store
D:\Python\Jupyter\testing\1733.jpg store
D:\Python\Jupyter\testing\1734.jpg Street
D:\Python\Jupyter\testing\1735.jpg Forest
D:\Python\Jupyter\testing\1736.jpg Street
D:\Python\Jupyter\testing\1737.jpg store
D:\Python\Jupyter\testing\1738.jpg Suburb
D:\Python\Jupyter\testing\1739.jpg store
D:\Python\Jupyter\testing\174.jpg Street
D:\Python\Jupyter\testing\1740.jpg industrial
D:\Python\Jupyter\testing\1741.jpg kitchen
D:\Python\Jupyter\testing\1742.jpg store
D:\Python\Jupyter\testing\1743.jpg store
D:\Python\Jupyter\testing\1744.jpg store
D:\Python\Jupyter\testing\1745.jpg store
D:\Python\Jupyter\testing\1746.jpg industrial
D:\Python\Jupyter\testing\1747.jpg Mountain
D:\Python\Jupyter\testing\1748.jpg Insidecity
D:\Python\Jupyter\testing\1749.jpg store
D:\Python\Jupyter\te

...

D:\Python\Jupyter\testing\2270.jpg store
D:\Python\Jupyter\testing\2271.jpg bedroom
D:\Python\Jupyter\testing\2272.jpg bedroom
D:\Python\Jupyter\testing\2273.jpg bedroom
D:\Python\Jupyter\testing\2274.jpg Street
D:\Python\Jupyter\testing\2275.jpg Forest
D:\Python\Jupyter\testing\2276.jpg TallBuilding
D:\Python\Jupyter\testing\2277.jpg store
D:\Python\Jupyter\testing\2278.jpg kitchen
D:\Python\Jupyter\testing\2279.jpg store
D:\Python\Jupyter\testing\228.jpg store
D:\Python\Jupyter\testing\2280.jpg store
D:\Python\Jupyter\testing\2281.jpg bedroom
D:\Python\Jupyter\testing\2282.jpg store
D:\Python\Jupyter\testing\2283.jpg OpenCountry
D:\Python\Jupyter\testing\2284.jpg store
D:\Python\Jupyter\testing\2285.jpg store
D:\Python\Jupyter\testing\2286.jpg TallBuilding
D:\Python\Jupyter\testing\2287.jpg Suburb
D:\Python\Jupyter\testing\2288.jpg Mountain
D:\Python\Jupyter\testing\2289.jpg store
D:\Python\Jupyter\testing\229.jpg Mountain
D:\Python\Jupyter\testing\2290.jpg industrial
D:\Python\Jupyt

D:\Python\Jupyter\testing\2455.jpg OpenCountry
D:\Python\Jupyter\testing\2456.jpg store
D:\Python\Jupyter\testing\2457.jpg Suburb
D:\Python\Jupyter\testing\2458.jpg Mountain
D:\Python\Jupyter\testing\2459.jpg Street
D:\Python\Jupyter\testing\246.jpg bedroom
D:\Python\Jupyter\testing\2460.jpg Forest
D:\Python\Jupyter\testing\2461.jpg store
D:\Python\Jupyter\testing\2462.jpg Mountain
D:\Python\Jupyter\testing\2463.jpg Highway
D:\Python\Jupyter\testing\2464.jpg Street
D:\Python\Jupyter\testing\2465.jpg Highway
D:\Python\Jupyter\testing\2466.jpg bedroom
D:\Python\Jupyter\testing\2467.jpg kitchen
D:\Python\Jupyter\testing\2468.jpg industrial
D:\Python\Jupyter\testing\2469.jpg store
D:\Python\Jupyter\testing\247.jpg kitchen
D:\Python\Jupyter\testing\2470.jpg kitchen
D:\Python\Jupyter\testing\2471.jpg Suburb
D:\Python\Jupyter\testing\2472.jpg Forest
D:\Python\Jupyter\testing\2473.jpg TallBuilding
D:\Python\Jupyter\testing\2474.jpg Insidecity
D:\Python\Jupyter\testing\2475.jpg livingroom
D:\Py

D:\Python\Jupyter\testing\2627.jpg bedroom
D:\Python\Jupyter\testing\2628.jpg Coast
D:\Python\Jupyter\testing\2629.jpg kitchen
D:\Python\Jupyter\testing\263.jpg Forest
D:\Python\Jupyter\testing\2630.jpg Forest
D:\Python\Jupyter\testing\2631.jpg kitchen
D:\Python\Jupyter\testing\2632.jpg bedroom
D:\Python\Jupyter\testing\2633.jpg store
D:\Python\Jupyter\testing\2634.jpg industrial
D:\Python\Jupyter\testing\2635.jpg store
D:\Python\Jupyter\testing\2636.jpg Forest
D:\Python\Jupyter\testing\2637.jpg Suburb
D:\Python\Jupyter\testing\2638.jpg bedroom
D:\Python\Jupyter\testing\2639.jpg industrial
D:\Python\Jupyter\testing\264.jpg store
D:\Python\Jupyter\testing\2640.jpg bedroom
D:\Python\Jupyter\testing\2641.jpg Mountain
D:\Python\Jupyter\testing\2642.jpg Mountain
D:\Python\Jupyter\testing\2643.jpg Highway
D:\Python\Jupyter\testing\2644.jpg industrial
D:\Python\Jupyter\testing\2645.jpg Mountain
D:\Python\Jupyter\testing\2646.jpg Coast
D:\Python\Jupyter\testing\2647.jpg kitchen
D:\Python\Jupyt

D:\Python\Jupyter\testing\2808.jpg Street
D:\Python\Jupyter\testing\2809.jpg store
D:\Python\Jupyter\testing\281.jpg Suburb
D:\Python\Jupyter\testing\2810.jpg Street
D:\Python\Jupyter\testing\2811.jpg Suburb
D:\Python\Jupyter\testing\2812.jpg Mountain
D:\Python\Jupyter\testing\2813.jpg store
D:\Python\Jupyter\testing\2814.jpg Mountain
D:\Python\Jupyter\testing\2815.jpg store
D:\Python\Jupyter\testing\2816.jpg Forest
D:\Python\Jupyter\testing\2817.jpg livingroom
D:\Python\Jupyter\testing\2818.jpg OpenCountry
D:\Python\Jupyter\testing\2819.jpg Street
D:\Python\Jupyter\testing\282.jpg TallBuilding
D:\Python\Jupyter\testing\2820.jpg store
D:\Python\Jupyter\testing\2821.jpg kitchen
D:\Python\Jupyter\testing\2822.jpg store
D:\Python\Jupyter\testing\2823.jpg Mountain
D:\Python\Jupyter\testing\2824.jpg store
D:\Python\Jupyter\testing\2825.jpg store
D:\Python\Jupyter\testing\2826.jpg Suburb
D:\Python\Jupyter\testing\2827.jpg kitchen
D:\Python\Jupyter\testing\2828.jpg Office
D:\Python\Jupyter\te

D:\Python\Jupyter\testing\299.jpg Street
D:\Python\Jupyter\testing\3.jpg industrial
D:\Python\Jupyter\testing\30.jpg store
D:\Python\Jupyter\testing\300.jpg Mountain
D:\Python\Jupyter\testing\301.jpg Insidecity
D:\Python\Jupyter\testing\302.jpg kitchen
D:\Python\Jupyter\testing\303.jpg store
D:\Python\Jupyter\testing\304.jpg Insidecity
D:\Python\Jupyter\testing\305.jpg Mountain
D:\Python\Jupyter\testing\306.jpg Coast
D:\Python\Jupyter\testing\307.jpg store
D:\Python\Jupyter\testing\308.jpg store
D:\Python\Jupyter\testing\309.jpg Forest
D:\Python\Jupyter\testing\31.jpg store
D:\Python\Jupyter\testing\310.jpg kitchen
D:\Python\Jupyter\testing\311.jpg industrial
D:\Python\Jupyter\testing\312.jpg Office
D:\Python\Jupyter\testing\313.jpg store
D:\Python\Jupyter\testing\314.jpg industrial
D:\Python\Jupyter\testing\315.jpg kitchen
D:\Python\Jupyter\testing\316.jpg Mountain
D:\Python\Jupyter\testing\317.jpg Office
D:\Python\Jupyter\testing\318.jpg store
D:\Python\Jupyter\testing\319.jpg OpenCo